<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [124]:
import re
import json

no_ascii_char = ['é', '‟', 'ø', 'š', '«', '¸', '…', '\xad', '¬', 'θ', ';', '\x9d', '∝', '´', '¾', '€', '÷', '′', '„', 'в', '£', 'â', 'ƞ', 'ã', '≠', 'œ', '¼', '∏', '¢', '🙂', '“', 'с', '’', '∠', 'т', 'π', '¤', '·', '™', '⁄', '�', 'ª', '—', '∪', '•', '₹', '³', '‘', '¥', '½', 'δ', '∩', '²', 'ō', 'ƫ', 'ў', 'º', 'ʼ', 'æ', '∆', '≥', 'е', '×', '¯', '∗', 'ç', '\x81', '√', 'ï', 'ˆ', '¹', '‚', 'ɵ', '−', 'í', '°', '≤', '–', '♠', '”']

def get_number_mapping(input_list):
    sym2num = {}
    num2sym = {}
    for num_i in input_list:
        if num_i not in sym2num:
            sym_i = 'n{}'.format(len(sym2num))
            sym2num[sym_i] = num_i
            num2sym[num_i] = sym_i
    return sym2num, num2sym

def is_ascii(s):
    res = []
    for c in s:
        if ord(c) >= 128:
            res.append(c)
    return res

In [144]:
def preprocess(data_version = 'train'):
    with open('./{}.json'.format(data_version), 'r') as f:
        data = json.load(f)
    print(len(data))
    with open('./{}.txt'.format(data_version), 'w') as g:
        to_write = []
        for item in data:
            problem = str(item['Problem']).lower()
            problem = problem.replace(',', ' , ').replace('(', ' ( ').replace(')', ' ) ').replace('|', ' | ').strip().strip('|').strip()
            problem = ' ' + " ".join(problem.split()) + ' '
            
            result = str(item['linear_formula']).lower()
            result = result.replace(',', ' , ').replace('(', ' ( ').replace(')', ' ) ').replace('|', ' | ').strip().strip('|').strip()
            result = ' ' + " ".join(result.split()) + ' '
            numbers = re.findall(r"\d+\.?\d*", problem)
            sym2num, num2sym = get_number_mapping(numbers)
            for num_i in num2sym:
                if ' '+num_i+' ' in problem:
                    problem = problem.replace(' '+num_i+' ', ' '+num2sym[num_i]+' ')
                else:
                    raise RuntimeError()
            problem = problem.strip()
            result = result.strip()
            for c in no_ascii_char:
                problem = problem.replace(c, 'no_ascii_{}'.format(no_ascii_char.index(c)))
                result = result.replace(c, 'no_ascii_{}'.format(no_ascii_char.index(c)))
            to_write.append((problem,result))
        to_write = sorted(to_write, key=lambda x:len(x[0].split()))
        print('writing...')
        for data_i in to_write:
            g.write(data_i[0]+'\t'+data_i[1]+'\n')

In [148]:
preprocess('train')

29837
writing...


In [149]:
preprocess('dev')
preprocess('test')

4475
writing...
2985
writing...


In [151]:
arr_len = []
with open('train.txt', 'r') as f:
    for l in f:
        arr_len.append(len(l.strip().split('\t')[1].split()))
        

13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
6
13
13
13
44
6
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
6
34
13
27
13
34
13
13
13
13
13
6
13
11
13
13
13
6
6
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
6
13
13
13
13
13
34
13
13
13
6
13
6
6
13
13
18
6
6
6
13
13
13
13
13
13
13
13
13
34
13
13
13
13
6
13
6
13
13
13
13
13
6
13
13
13
13
13
13
13
6
34
18
6
13
13
27
6
34
27
13
11
34
34
6
13
13
13
6
11
13
11
34
13
13
13
6
27
6
6
13
34
13
13
90
34
13
6
6
41
13
34
27
13
13
34
18
34
41
6
6
34
13
13
34
34
13
34
13
6
6
13
6
6
13
13
6
13
6
34
34
13
6
13
13
6
4
11
18
6
13
18
13
13
34
6
13
34
6
13
18
6
13
20
90
13
13
6
34
13
20
20
20
6
13
13
13
27
20
13
34
6
20
6
11
13
90
20
90
13
34
13
20
34
13
18
90
13
27
13
6
13
20
6
27
13
13
6
13
20
13
6
20
13
13
16
13
13
34
90
6
4
27
13
34
6
13
6
42
13
6
13
6
6
13
90
6
20
20
13
20
6
34
13
13
13
13
6
6
13
13
90
13
13
20
16
6
27
20
27
13
13
20
13
20
6
13
13
13
20
6
34
13
6
42
20
13
6
13
20
6
34
41
13
20
6
4
20
20
34
67
18
18
67
18


11
11
13
13
37
20
20
48
20
6
27
6
27
41
20
20
27
13
20
90
27
41
49
44
20
27
34
20
34
27
42
13
20
55
6
6
27
27
20
20
6
13
13
13
25
37
27
20
20
20
41
27
13
20
69
41
27
34
6
6
41
34
34
13
27
13
34
6
23
13
6
90
13
27
27
13
20
27
34
13
35
90
13
30
25
27
13
13
34
49
4
42
23
25
25
6
34
37
27
13
34
42
20
13
27
60
13
41
13
90
20
90
27
48
20
30
11
41
90
6
13
34
48
13
20
37
13
48
20
20
23
13
34
20
6
20
25
25
42
34
27
41
27
27
27
6
48
20
6
76
13
20
34
20
76
20
13
13
20
20
20
34
20
6
13
48
20
6
48
6
13
41
62
27
25
20
34
20
41
6
13
20
13
20
6
6
65
20
20
13
13
20
20
13
27
20
27
13
27
55
23
18
6
6
23
13
25
37
6
13
34
20
6
13
6
34
39
13
20
13
27
13
76
6
20
32
34
20
13
44
13
23
20
13
6
13
6
13
27
6
27
13
41
20
41
20
90
20
32
27
35
32
13
76
37
48
13
13
41
20
13
20
18
13
39
13
48
34
20
13
13
20
20
4
25
13
27
34
6
18
13
34
20
13
18
13
48
6
13
20
6
27
48
13
27
23
20
48
13
27
13
32
20
49
4
34
41
13
6
13
8
20
20
13
13
32
48
41
13
25
13
13
20
13
20
37
20
41
18
13
20
27
37
20
6
20
13
49
27
37
48
6
27
18
37
48
1

49
27
27
48
39
13
34
27
6
32
13
6
20
13
6
13
34
41
20
6
62
13
49
48
20
20
13
6
20
20
27
6
13
27
20
6
11
27
27
27
35
20
20
6
69
13
44
13
20
6
6
13
20
20
20
34
6
20
27
20
48
13
27
20
20
13
6
13
34
6
27
6
13
6
67
27
13
13
6
32
6
35
35
48
27
13
13
20
13
48
34
27
34
62
27
31
34
6
41
6
48
27
20
27
13
34
20
13
27
27
40
6
13
41
27
6
6
20
62
13
20
13
13
6
20
34
6
27
13
13
13
90
6
62
27
31
20
27
27
27
41
20
6
31
20
27
20
6
41
13
13
55
13
13
13
20
20
20
34
13
13
4
20
27
13
20
20
6
6
35
13
20
20
41
20
13
34
20
34
13
46
27
6
13
20
20
30
27
20
62
48
6
48
13
20
62
6
13
27
27
20
6
8
6
6
13
20
20
30
20
13
34
30
41
20
20
20
13
34
27
27
13
48
20
49
20
6
20
13
13
20
13
28
13
13
20
27
27
6
62
6
34
8
48
39
48
27
35
20
27
20
13
34
6
34
55
20
74
13
20
20
90
20
20
34
13
20
6
20
13
27
6
20
13
13
13
8
20
41
20
20
20
48
8
13
20
27
6
13
34
6
27
20
20
62
48
27
20
20
20
74
13
31
20
32
27
27
6
20
40
34
13
20
27
6
20
49
13
20
27
6
20
6
74
34
13
27
20
34
13
41
48
20
27
34
6
6
27
34
48
27
20
49
34
20
27
27
27
13
13
48
1

6
13
20
41
20
13
27
41
20
20
27
20
27
27
20
27
20
6
41
20
41
41
20
20
27
35
27
46
20
34
6
20
20
34
41
34
13
35
6
13
46
20
62
27
20
13
27
13
41
13
20
6
20
41
13
20
13
20
41
20
41
20
118
34
42
44
13
27
13
20
27
62
6
41
20
11
13
76
13
41
27
4
48
20
20
20
20
20
13
90
34
6
46
48
25
20
41
27
27
34
13
41
13
6
41
20
6
41
6
20
20
32
37
62
48
13
34
34
41
13
35
27
27
27
18
20
68
27
20
41
34
46
20
34
13
41
34
13
20
20
41
27
20
34
53
41
20
20
27
34
20
41
41
20
6
20
20
27
27
34
13
27
13
18
44
20
48
20
20
41
20
118
27
20
27
13
27
20
46
20
34
48
34
27
18
20
13
27
42
65
20
18
44
20
20
20
41
13
62
48
37
27
76
6
6
34
13
6
20
20
34
20
16
32
32
48
27
41
20
27
13
20
118
13
6
20
23
20
18
42
20
62
18
20
13
25
13
32
48
20
20
44
41
27
27
27
6
6
27
34
48
18
25
20
13
20
20
20
20
62
27
27
20
46
20
6
6
13
20
34
20
46
13
13
20
27
20
27
48
20
20
20
20
48
20
34
13
13
20
13
20
20
27
46
20
11
68
6
6
13
27
20
34
13
27
13
20
13
27
13
20
13
27
13
20
32
27
34
55
34
44
20
28
27
41
27
44
13
13
18
20
20
27
27
20
13
27
18
62
34

13
139
41
27
48
20
13
27
55
6
13
34
13
46
39
76
27
13
55
27
20
41
62
34
13
20
20
34
41
58
20
46
27
6
41
20
69
27
27
20
20
35
27
111
13
6
6
34
20
20
20
48
20
20
6
27
13
20
41
34
20
34
46
6
20
34
6
62
13
13
55
16
13
27
27
34
20
20
30
48
13
41
13
39
20
27
20
41
48
20
18
20
13
46
20
46
20
48
18
25
34
6
27
55
18
41
20
34
34
41
20
41
34
27
13
6
48
34
34
20
20
27
34
55
34
41
20
48
27
76
41
20
20
18
20
6
48
20
34
27
53
41
13
48
34
13
41
20
48
46
20
39
13
48
34
20
20
27
20
6
6
46
34
41
41
41
35
20
55
41
35
20
20
6
20
20
41
20
6
48
13
20
20
34
34
48
69
27
74
13
13
20
20
20
13
27
13
48
16
34
41
13
34
20
34
20
34
20
20
48
49
34
6
76
46
46
48
6
11
41
97
34
46
20
20
48
13
20
6
55
30
13
20
30
13
48
55
34
55
34
20
48
20
46
20
13
20
13
55
20
111
13
20
111
46
41
48
20
20
46
13
20
27
46
27
13
76
13
34
51
6
48
34
34
13
20
11
48
41
20
20
39
32
27
34
20
48
20
6
20
30
6
20
20
27
34
6
20
27
34
20
20
27
20
27
6
48
13
34
46
48
20
27
34
13
69
41
6
18
27
34
18
34
20
55
39
20
48
20
62
13
27
20
20
20
20
20
13
6
34


...

20
62
20
55
27
13
34
48
18
42
55
34
27
48
48
34
20
27
6
6
48
27
20
62
20
37
27
20
27
27
48
41
27
48
27
6
34
27
34
20
27
20
27
6
27
34
62
27
20
27
34
20
27
27
27
27
27
39
20
13
41
55
34
20
69
27
27
34
20
20
13
13
34
27
27
13
34
27
48
55
6
18
27
13
20
69
13
48
20
20
6
20
20
6
62
41
27
41
27
27
27
20
48
55
34
13
20
48
41
34
76
34
34
48
41
13
6
34
48
34
27
44
6
48
44
27
27
18
6
20
34
41
55
13
55
20
34
48
6
13
76
34
27
13
20
13
34
42
20
13
27
20
48
69
55
20
34
69
27
20
55
20
20
41
25
27
20
62
27
6
20
27
13
34
20
20
27
27
34
13
27
27
20
34
13
48
27
20
20
55
34
41
20
20
20
13
62
6
13
6
76
20
27
34
48
13
34
13
13
13
27
20
41
6
62
20
13
41
27
44
27
27
27
20
27
27
34
41
20
18
55
6
6
20
48
27
20
44
20
44
16
18
6
13
27
27
34
27
41
34
41
20
41
27
44
41
20
55
27
48
34
48
76
27
41
27
34
34
6
41
44
62
62
48
20
20
48
34
27
27
27
20
18
13
11
13
34
27
44
27
20
27
20
13
20
48
20
16
27
27
55
41
76
20
48
27
48
6
48
48
20
20
20
6
41
55
6
27
18
48
20
48
13
48
20
62
34
20
20
25
20
27
27
20
48
20
20
13
13
20
41

27
27
34
55
55
34
27
13
6
20
27
34
60
20
27
20
20
34
34
20
27
48
27
34
6
20
34
34
20
13
13
27
13
6
27
102
20
34
34
11
48
13
37
20
20
13
34
20
41
6
25
34
34
55
20
102
13
48
27
27
55
41
41
34
6
48
20
34
76
41
20
20
13
48
34
20
13
20
34
34
34
6
55
20
34
20
20
27
27
20
20
76
76
27
20
48
55
41
34
48
27
48
34
20
27
34
104
20
27
20
34
6
13
6
48
69
13
20
41
6
20
34
48
34
20
48
13
34
34
6
80
13
48
27
74
27
6
69
25
20
48
20
27
32
34
20
34
104
55
32
20
13
13
41
102
48
69
13
48
48
41
48
34
6
27
20
6
69
27
34
20
20
69
116
20
48
27
13
13
48
48
27
13
27
20
102
34
13
39
104
13
34
34
20
6
32
20
41
62
32
27
20
41
13
20
41
20
76
34
13
34
25
13
48
20
34
34
32
34
13
27
27
27
20
48
48
13
20
27
34
27
20
62
20
27
20
48
37
39
27
34
34
27
25
20
34
21
39
34
20
34
34
20
27
25
34
20
6
41
27
13
34
6
69
6
32
34
76
48
74
25
55
48
6
20
13
48
48
76
76
34
20
48
25
34
55
34
13
20
39
34
27
6
32
34
41
74
34
55
39
48
20
48
20
34
13
27
34
20
41
104
74
69
27
20
13
20
34
6
34
76
55
34
20
20
130
20
41
20
48
20
6
20
27
20
34
20


48
20
48
6
48
27
20
48
90
27
13
83
62
41
48
27
13
27
41
90
62
34
13
62
48
48
27
27
13
20
27
13
55
41
48
6
55
34
48
55
6
27
27
27
20
27
55
48
39
13
27
34
48
34
41
48
32
20
34
13
48
27
62
41
34
90
34
27
41
41
23
55
48
13
69
48
48
20
83
13
34
13
27
48
48
41
48
34
27
55
55
13
27
34
46
55
13
34
41
6
34
48
41
41
48
6
34
84
55
41
34
13
6
48
48
90
13
34
41
55
27
20
55
69
20
34
104
13
13
34
34
16
32
27
34
41
46
27
27
20
104
41
41
83
34
20
34
20
13
55
20
11
34
13
67
20
13
34
34
34
34
41
32
48
62
13
41
34
13
41
20
27
13
34
20
34
34
62
20
34
55
27
13
13
34
13
34
34
13
41
48
20
48
20
13
13
13
13
20
55
84
13
13
55
20
20
34
55
34
42
20
42
13
20
41
34
20
48
41
20
27
48
55
13
6
41
41
41
13
41
55
27
6
48
27
13
13
41
27
20
13
13
27
34
34
20
34
62
18
55
20
27
48
20
13
20
34
62
34
13
62
13
48
34
34
74
104
74
13
20
6
27
41
48
27
41
20
20
69
48
48
13
48
6
20
34
20
34
83
20
34
34
55
41
48
34
20
34
41
48
6
20
34
46
27
48
48
13
20
34
69
27
27
74
74
20
13
69
20
27
6
18
34
55
41
69
48
34
42
6
18
20
41
20
13
20
20

125
34
41
27
55
20
48
6
27
27
41
6
13
41
13
37
6
20
34
6
62
62
41
6
6
70
41
55
6
70
34
13
11
6
41
39
41
11
48
20
27
70
6
160
20
6
48
20
27
41
39
69
48
27
20
27
41
27
20
20
48
34
41
62
34
41
13
104
27
34
13
55
90
13
41
27
13
6
34
20
41
13
13
83
34
27
90
62
62
13
55
34
34
104
20
13
55
27
65
20
58
13
13
34
55
65
76
27
55
41
34
27
13
13
55
13
62
27
27
55
20
55
27
69
76
27
41
13
27
13
62
13
41
41
34
41
20
20
13
6
62
55
27
48
76
55
69
27
41
41
20
41
41
62
65
27
27
13
55
34
34
65
27
55
41
41
46
27
27
41
13
34
65
55
13
27
20
13
20
6
39
27
67
90
58
13
6
48
39
39
6
55
41
90
41
13
41
39
76
41
13
55
62
41
27
39
34
41
34
13
6
65
13
41
41
6
27
41
41
13
13
76
65
34
90
27
13
13
27
55
27
48
41
41
34
27
18
13
41
41
41
20
13
76
13
13
41
13
13
6
13
76
55
41
6
48
34
34
6
48
76
41
41
41
69
13
20
48
55
13
13
69
34
13
13
63
6
27
111
20
41
97
113
34
13
27
62
13
27
48
13
13
53
48
69
13
53
62
41
6
6
13
13
69
34
13
6
13
55
27
20
69
27
6
13
34
20
69
58
48
13
48
104
34
6
41
6
34
27
111
20
13
34
69
13
20
13
32
55
34

20
20
20
20
34
41
20
41
20
13
25
41
41
41
41
41
41
20
20
34
20
20
90
34
55
13
6
6
11
6
49
83
83
83
34
34
34
13
41
62
34
27
6
34
55
13
